In [1]:
import pandas as pd

def aggregate_monthly_data(data):
    # Load the CSV file
    #file_path = f'{spreadsheet_name}'
    #data = pd.read_csv(file_path)

    # Convert the 'timestamp' column to datetime
    data['timestamp'] = pd.to_datetime(data['timestamp'])

    # Set the timestamp as the index
    data.set_index('timestamp', inplace=True)

    # Aggregate the energy consumption data monthly
    monthly_energy_aggregates = data.filter(like='out.').resample('M').sum()

    # Aggregate non-changing columns by taking the first available value for each month
    non_changing_columns = data[['upgrade', 'in.state', 'in.comstock_building_type', 'models_used', 'floor_area_represented']].resample('M').first()

    # Combine the aggregated data
    monthly_aggregated_data = pd.concat([non_changing_columns, monthly_energy_aggregates], axis=1)

    # Reset index to make the timestamp a column again
    monthly_aggregated_data.reset_index(inplace=True)

    return monthly_aggregated_data



In [2]:
# Example usage
#baseline_data = 'up00-1a-smalloffice.csv'
#baseline_monthly_df = aggregate_monthly_data(baseline_data)

#LED_data = 'up05-1a-smalloffice.csv'
#LED_monthly_df = aggregate_monthly_data(LED_data)
#baseline_monthly_df['out.electricity.total.energy_consumption.kwh']
#LED_monthly_df['out.electricity.total.energy_consumption.kwh']

In [3]:
#baseline_monthly_df['out.electricity.total.energy_consumption.kwh'] - LED_monthly_df['out.electricity.total.energy_consumption.kwh']
#LED_monthly_df['out.electricity.total.energy_consumption.kwh.savings']

In [4]:
state_abv = {
    '2': 'AK',
    '1': 'AL',
    '5': 'AR',
    '4': 'AZ',
    '6': 'CA',
    '8': 'CO',
    '9': 'CT',
    '11': 'DC',
    '10': 'DE',
    '12': 'FL',
    '13': 'GA',
    '15': 'HI',
    '19': 'IA',
    '16': 'ID',
    '17': 'IL',
    '18': 'IN',
    '20': 'KS',
    '21': 'KY',
    '22': 'LA',
    '25': 'MA',
    '24': 'MD',
    '23': 'ME',
    '26': 'MI',
    '27': 'MN',
    '29': 'MO',
    '28': 'MS',
    '30': 'MT',
    '37': 'NC',
    '38': 'ND',
    '31': 'NE',
    '33': 'NH',
    '34': 'NJ',
    '35': 'NM',
    '32': 'NV',
    '36': 'NY',
    '39': 'OH',
    '40': 'OK',
    '41': 'OR',
    '42': 'PA',
    '44': 'RI',
    '45': 'SC',
    '46': 'SD',
    '47': 'TN',
    '48': 'TX',
    '49': 'UT',
    '51': 'VA',
    '50': 'VT',
    '53': 'WA',
    '55': 'WI',
    '54': 'WV',
    '56': 'WY'}

In [5]:
test_state_abv = {
    '2': 'AK',
    '1': 'AL',
    '5': 'AR',
    '4': 'AZ'}

In [6]:
comstock_buildings = ['quickservicerestaurant',
                      'fullservicerestaurant',
                      'smalloffice',
                      'mediumoffice',
                      'largeoffice',
                      'warehouse',
                      'smallhotel',
                      'largehotel',
                      'outpatient',
                      'hospital',
                      'secondaryschool',
                      'primaryschool',
                      'retailstandalone',
                      'retailstripmall']

In [7]:
test_comstock_buildings = ['smalloffice']

In [8]:
upgrades = ['17','18']
test_upgrades = ['17']

In [9]:
import requests
from io import StringIO

In [10]:
base_url = "https://oedi-data-lake.s3.amazonaws.com/nrel-pds-building-stock/end-use-load-profiles-for-us-building-stock/2023/comstock_amy2018_release_2/timeseries_aggregates/by_state/upgrade={UP}/state={STATE}/up{up}-{state}-{BUILDING}.csv"
#https://oedi-data-lake.s3.amazonaws.com/nrel-pds-building-stock/end-use-load-profiles-for-us-building-stock/2023/comstock_amy2018_release_2/timeseries_aggregates/by_state/upgrade=17/state=CO/up17-co-retailstandalone.csv

In [11]:
# Initialize a blank DataFrame with specific columns
data_df = pd.DataFrame(columns=['upgrade', 'state', 'comstock_building_type', 'floor_area_represented', 'sum.electricity.total.energy_consumption.kwh.savings', 'sum.electricity.total.energy_consumption.kwh', 'sum.natural_gas.total.energy_consumption.kwh.savings', 'sum.natural_gas.total.energy_consumption.kwh'])


In [12]:
# loop over upgrades
for upgrade in test_upgrades:
    UP = str(upgrade)
    up = f"{upgrade:02}"  # Ensures leading zero for single-digit numbers

    # Loop through each state abbreviation
    for state in state_abv.values():
        
        #loop over buildings
        for building in test_comstock_buildings:
            
            # Construct the URL for the current state
            url = base_url.format(UP=UP, up=up, STATE=state.upper(), state=state.lower(), BUILDING=building)

            try:
                # Make a GET request to fetch the CSV content
                response = requests.get(url)
                if response.status_code == 200:
                    print(f"DOWNLOADING upgrade: {up}, state: {state}, building_type: {building}")
                    # Convert the CSV content to a DataFrame
                    csv_content = StringIO(response.content.decode('utf-8'))
                    df = pd.read_csv(csv_content)

                    monthly_df = aggregate_monthly_data(df)
                    electricity_savings = monthly_df['out.electricity.total.energy_consumption.kwh.savings'].sum()
                    electricity_upgrade = monthly_df['out.electricity.total.energy_consumption.kwh'].sum()
                    gas_savings = monthly_df['out.natural_gas.total.energy_consumption.kwh.savings'].sum()
                    gas_upgrade = monthly_df['out.natural_gas.total.energy_consumption.kwh'].sum()
                    floor_area_represented = monthly_df['floor_area_represented'].unique()[0]
                    building_type = monthly_df['in.comstock_building_type'].unique()[0]
                    print(f"RESULTS upgrade: {up}, state: {state}, building_type: {building_type}, floor_area_represented: {floor_area_represented:.2f}, electricity_savings: {electricity_savings:.2f}, electricity_upgrade: {electricity_upgrade:.2f}, gas_savings: {gas_savings:.2f}, gas_upgrade: {gas_upgrade:.2f}")
                 
                    new_data = {
                        'upgrade': [up],
                        'state': [state.upper()],
                        'comstock_building_type': [building_type],
                        'floor_area_represented': [floor_area_represented],
                        'sum.electricity.total.energy_consumption.kwh.savings': [electricity_savings],
                        'sum.electricity.total.energy_consumption.kwh': [electricity_upgrade],
                        'sum.natural_gas.total.energy_consumption.kwh.savings': [gas_savings],
                        'sum.natural_gas.total.energy_consumption.kwh': [gas_upgrade]
                    }

                    # add new entry to DF
                    new_row_df = pd.DataFrame(new_data)
                    data_df = pd.concat([data_df, new_row_df], ignore_index=True)
                    
                else:
                    print(f"Failed to download data for {state.upper()}: HTTP {response.status_code}")
            except Exception as e:
                print(f"Error downloading data for {state.upper()}: {e}")
                
print('saving data.csv')                
data_df.to_csv('data.csv', index=False)

DOWNLOADING upgrade: 17, state: AK, building_type: smalloffice
RESULTS upgrade: 17, state: AK, building_type: SmallOffice, floor_area_represented: 8405165.35, electricity_savings: 2995784.22, electricity_upgrade: 128200487.93, gas_savings: 55889353.16, gas_upgrade: 6976157.89
DOWNLOADING upgrade: 17, state: AL, building_type: smalloffice
RESULTS upgrade: 17, state: AL, building_type: SmallOffice, floor_area_represented: 66424644.42, electricity_savings: 214479048.45, electricity_upgrade: 803068340.57, gas_savings: 48646628.22, gas_upgrade: 30458860.47
DOWNLOADING upgrade: 17, state: AR, building_type: smalloffice
RESULTS upgrade: 17, state: AR, building_type: SmallOffice, floor_area_represented: 45120375.53, electricity_savings: 124601035.36, electricity_upgrade: 552682253.25, gas_savings: 82072921.55, gas_upgrade: 38968249.90
DOWNLOADING upgrade: 17, state: AZ, building_type: smalloffice
RESULTS upgrade: 17, state: AZ, building_type: SmallOffice, floor_area_represented: 78070036.32, e